In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
test = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
sub = pd.read_csv('../input/health-insurance-cross-sell-prediction/sample_submission.csv')


In [ ]:
train.head()

In [ ]:
train.describe().T

In [ ]:
sns.countplot(train['Response'])

Distribution of Insured vs Non Insured

In [ ]:
sns.countplot(train['Previously_Insured'])

Let's check the distribution of Policy Buyer vs non Buyer

In [ ]:
sns.countplot(train['Response'])

Check for any missing values

In [ ]:
train.isnull().sum()

Check for Male/Female Dsitribution.

In [ ]:
sns.countplot(train['Gender'])

Let's Convert Sex in categorical values

In [ ]:
gender = pd.get_dummies(train['Gender'])
train=pd.concat([train,gender], axis=1)
train = train.drop('Gender', axis=1)

In [ ]:
train['Vehicle_Age'].unique()

Let's conver Vehicle Age into categorical values

In [ ]:
train = pd.concat([train,(pd.get_dummies(train['Vehicle_Age']))],axis=1)
train.drop('Vehicle_Age', axis=1, inplace=True)

In [ ]:
train.set_index('id', inplace=True)

In [ ]:
train.head()

In [ ]:
train = pd.concat([train,pd.get_dummies(train['Vehicle_Damage'])], axis=1)
train.drop('Vehicle_Damage', axis=1, inplace=True)

In [ ]:
sns.countplot(train['Driving_License'])

Only 812 People don't have Driving Liscense

In [ ]:
train[train['Driving_License']==0]

Droping Male coulmn since it is just opposite of female

In [ ]:
train = train.drop(['Male'], axis=1)

Vehicle Damage yes and no is just opposite so, both of them is not required.

In [ ]:
train = train.drop(['No'], axis=1)

Out 3 age column one of them is not required.

In [ ]:
train = train.drop(['< 1 Year'], axis=1)

In [ ]:
sns.boxenplot(data=train, x='Response',y='Annual_Premium')

In [ ]:
train['Annual_Premium'].hist()

Looks like more then Rs 120000 premiums are outliers hne removing them

In [ ]:
train=train[train['Annual_Premium']<=120000]

In [ ]:
sns.boxenplot(data=train, x='Response',y='Policy_Sales_Channel')

In [ ]:
sns.boxplot(data=train, x='Response',y='Vintage')

In [ ]:
sns.boxplot(data=train, x='Response',y='Previously_Insured')

In [ ]:
sns.countplot(train[train['Response']==1]['Previously_Insured'])

In [ ]:
sns.countplot(train[train['Response']==0]['Previously_Insured'])

In [ ]:
sns.boxenplot(data=train, x='Response',y='Region_Code')

In [ ]:
(train['Region_Code'].unique())

In [ ]:
sns.boxplot(data=train, x='Response',y='Age')

In [ ]:
sns.countplot(train[train['Driving_License']==0]['Response'])

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train.corr(), cmap='viridis', annot=True)

Deleting Driving Liscense since it has only 1% correlation with response and data is highly uneven.
Similarly deleting the Region_Code and vintage.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [ ]:
#train=train.drop('Policy_Sales_Channel', axis=1) This did not help
train=train.drop('Driving_License', axis=1) # Deleting this column gives better results
train=train.drop('Region_Code', axis=1)
train=train.drop('Vintage', axis=1) # Deleting this column gives yet better results for class1, hence this is better model.
train=train.drop('Annual_Premium', axis=1) # This have improved the perfromance dramatically 

In [ ]:
train.head()

In [ ]:
X = train.drop('Response', axis=1)
y = train['Response']

Since Response distribution is highly unevn we need to balance them with help of over srampling or undersampling. imblearn is the python module help us to do that.

In [ ]:

from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

samplers = [SMOTE(random_state=0), SMOTEENN(0.62,random_state=0), SMOTETomek(random_state=0)]
X_res, y_res = samplers[0].fit_resample(X, y)

X_comb, y_comb = samplers[1].fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_comb, y_comb, test_size=0.25, random_state=42)
scaller = MinMaxScaler()

In [ ]:
scalled_X_train = scaller.fit_transform(X_train)
rf = RandomForestClassifier()
rf.fit(scalled_X_train, y_train.values)
scalled_X_test = scaller.transform(X_test)
pred = rf.predict(scalled_X_test)

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
print(confusion_matrix(pred, y_test)) # Results after deleting DL and region Code, class 0 and class1 

WOW it predicts with 99% accuracy with Random Forest , lets try with xgboost, linearsvc and ANN as well.

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier()
model.fit(scalled_X_train, y_train.values)

In [ ]:
pred = model.predict(scalled_X_test)

In [ ]:
print(confusion_matrix(pred, y_test))

In [ ]:
print(classification_report(y_test, pred))

Extream Gradient Booost aslo give 99% but Random forest does  better when we compare with confusion matrix.

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC()
model.fit(scalled_X_train, y_train)
pred = model.predict(scalled_X_test)

In [ ]:
print(confusion_matrix(pred, y_test))

In [ ]:
print(classification_report(y_test, pred))

LinearSVC give 91% accuracy

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

In [ ]:

model.fit(x=scalled_X_train, y=y_train, epochs=50, validation_data=(scalled_X_test, y_test))

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.plot()

In [ ]:
pred = model.predict(scalled_X_test)

In [ ]:
pred = np.round(pred)

In [ ]:
print(confusion_matrix(pred, y_test))

In [ ]:
print(classification_report(y_test, pred))

It appears randon forest gives the best performance.